In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RouterChain, LLMChain

In [5]:
# Different Prompt Template
math_template =""" You are a helpful mathematician. You are good in solving the math problem and explain the solution in step by step.
                 Help to solve the below input maths problem.
                 \n\n
                 {input}"""

science_template =""" You are a helpful scientist. You are good in solving the science problem and explain back in step by step.
                 Help to solve the below input science problem.
                 \n\n
                 {input}"""
computer_template =""" You are a helpful computer programmer. You are good in solving the computer problem and explain with a sample program.
                 Help to solve the below input computer problem.
                 \n\n
                 {input}"""
history_template =""" You are a helpful history teacher. You are good in explaining the history event.
                 Help with the below input history question.
                 \n\n
                 {input}"""

In [60]:
prompt_infos = [
{
    "name":"science",
    "description":"Good for answering questions about science",
    "prompt_template":science_template
}, 
{
    "name":"history",
    "description":"Good for answering questions about history",
    "prompt_template":history_template
}, 
{
    "name":"math",
    "description":"Good for answering questions about maths problems",
    "prompt_template":math_template
},
{ 
    "name":"computerscience",
    "description": "Good for answering questions about computer science",
    "prompt_template":computer_template
}
]

In [9]:
from langchain.chains.router import MultiPromptChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser


In [10]:
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.5)


In [61]:
destination_chains ={}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template = prompt_template)
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = llm_chain

    destinations =[f"{p['name']}: {p['description']}" for p in prompt_infos]
    destinations_str = "\n".join(destinations)
   

In [62]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [77]:
Multi_prompt_Router_Template =""" Given a raw text input to a language model select a appropriate model prompt that best suite for. 
Feel free to revise the original input if the input need revising for better response from language model.
<<FORMATTING>>
Return markdown code snippet with a json object formatted to look like
```json
{{{{
    "destination" : string  name of the prompt used,
    "next_inputs" : string  modified version of original input 
}}}}
```

Remember "destinations" must be one of the candidate prompt specified below or it wil be "Default"
<<END_OF_FORMATTING>>
{destinations}
<<Input>>
{{input}}

<< Output (remember to include the markdown code block and ```json) >>"""


In [78]:
router_template = Multi_prompt_Router_Template.format(
    destinations = destinations_str
)
router_prompt = PromptTemplate(
    input_variables=["input"],
    template=router_template,
    output_parser= RouterOutputParser()
)

In [79]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [80]:
chain = MultiPromptChain(router_chain= router_chain,
                         destination_chains=destination_chains,
                         default_chain= default_chain,
                         verbose= True)

In [67]:
chain.run("what is 2+2?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is the sum of 2 and 2?'}
> Finished chain.


'To solve the problem of finding the sum of 2 and 2, we can follow these simple steps:\n\n1. **Identify the Numbers**: We have two numbers to add: 2 and 2.\n\n2. **Set Up the Addition**: We can write this as:\n   \\[\n   2 + 2\n   \\]\n\n3. **Perform the Addition**: Now, we add the two numbers together:\n   - Start with the first number, which is 2.\n   - Then, add the second number, which is also 2.\n   - So, we calculate:\n   \\[\n   2 + 2 = 4\n   \\]\n\n4. **Conclusion**: The sum of 2 and 2 is 4.\n\nThus, the final answer is:\n\\[\n\\text{The sum of 2 and 2 is } 4.\n\\]'

In [68]:
chain.run("what this command will do: python3 -m venv venv")



> Entering new MultiPromptChain chain...
computerscience: {'input': "What does the command 'python3 -m venv venv' do?"}
> Finished chain.


"The command `python3 -m venv venv` is used to create a virtual environment in Python. Let's break down what each part of the command does:\n\n1. **`python3`**: This specifies that you want to use Python 3 to run the command. If you have multiple versions of Python installed, this ensures that you're using Python 3.\n\n2. **`-m venv`**: The `-m` flag tells Python to run a module as a script. In this case, the module is `venv`, which is responsible for creating virtual environments.\n\n3. **`venv`**: This is the name of the directory that will be created to hold the virtual environment. You can name it anything you like, but it's common to name it `venv` or `.venv`.\n\n### What is a Virtual Environment?\n\nA virtual environment is an isolated environment in which you can install packages and dependencies for a specific project without affecting the global Python installation or other projects. This is especially useful when different projects require different versions of libraries.\n\n

In [69]:
chain.run("US Independance Day")



> Entering new MultiPromptChain chain...
history: {'input': 'What is the significance of US Independence Day?'}
> Finished chain.


"US Independence Day, celebrated on July 4th, marks the adoption of the Declaration of Independence in 1776, when the thirteen American colonies declared their independence from British rule. The significance of this day can be understood through several key points:\n\n1. **Foundational Moment**: Independence Day represents the birth of the United States as a sovereign nation. The Declaration of Independence articulated the colonies' desire to break free from British rule and established the principles of individual liberty and government by consent.\n\n2. **Symbol of Freedom**: The day symbolizes the struggle for freedom and democracy. It reflects the values of liberty, equality, and the right to self-governance, which have inspired many other nations and movements around the world.\n\n3. **Civic Identity**: Independence Day fosters a sense of national identity and unity among Americans. Celebrations often include parades, fireworks, and community events, reinforcing a shared history 

In [81]:
chain.run("How many players will play in cricket?")



> Entering new MultiPromptChain chain...
None: {'input': 'How many players are there in a cricket team?'}
> Finished chain.


'A cricket team consists of 11 players.'